### ADC,TDC信号条件选择
#### 中子飞行时间谱测量
物理过程：束流轰击靶以一定几率产生带电粒子，n，gamma等粒子。假设n，gamma的产生率为eff，其中gamma的比例为rgamma，n,gamma都可以被中子探测器探测(ng=0/1:gamma/neutron)。重带电粒子碎片可以被靶后Diple magnet偏走，而部分轻粒子没有被偏走，进入中子探测器，这部分占束流的比例为rcparticle(ng=2)。其余的事件，即比例为1-ngeff-cpeff的事件在探测器上不产生信号(ng=3)。

在下面的代码中，ng=0,1的事件的time、energy在TDC，ADC的量程范围内； ng=2的事件的time在TDC的量程范围内，部分事件超出ADC的量程；ng=3的事件在ADC的gata范围内的幅度为ADC基线(称为pedestal)和噪声的叠加，在TDC的量程范围内没有stop信号，使得TDC信号超界。

为了模拟实际情况10%的u或t只有一侧事件信号(TDC 超界，ADC=pedestal)

#### ADC/QDC 的pedestal 
![](pedestal.png)



``` cpp

// 将下列代码保存到treeoverflow.cc
// 在ROOT环境中运行：
// .L treeoverflow.cc
// 生成 treeover.root 文件
void treeoverflow(){
 const Double_t nD=500;//cm, distance between target and the scin.(Center)
 const Double_t nL=100;//cm, half length of the scin.
 const Double_t nTh=5;//cm, thickness of the scin.
 const Double_t ts=1./2.36;//ns, sigma of time of the scin.
 const Double_t lambda=380;//cm, attenuation lenght of the scin.
 const Double_t Qs=0.1;//relative energy resolution of the scin. 
 const Double_t vsc=7.5;//ns/cm, speed of light in the scin.

 //neutron & gamma
 const Double_t En0=50;//MeV, average neutron energy
 const Double_t Ens=10;//MeV, sigma of En
 const Double_t Eg0=2;//MeV, gamma energy  

 const Double_t ngeff=0.5;//=Nn,g/Nbeam， 1-eff = ratio of events with no real TDC&ADC signal: 
 const Double_t rgamma=0.1;//ratio of gamma, neutron=0.9; 

 //charged particle hited in Neutron Detector, 
 const Double_t cpeff=0.1;//ratio of charge particle
 const Double_t Ec0=100;//MeV
 const Double_t Ecs=5;//MeV

 //ADC
 const Double_t ADCgain=60;//1MeV=60ch.
 const Double_t ADCuPed=100;//baseline of ADC for upper side
 const Double_t ADCdPed=120;//                       for bottom side
 const Double_t ADCnoise=10;//sigma of noise
 const Double_t ADCoverflow=5000.;

 //TDC
 const Double_t TDCgain=40;//1ns=40ch.
 const Double_t TDCoverflow=5000;
  
  TFile *opf=new TFile("treeover.root","recreate");//新文件tree.root的指针 *opf
  TTree *opt=new TTree("tree","tree structure");//新tree的指针 *opt
  // 在tree结构中定义需要的变量分支
  Double_t x;
  Double_t E;
  Double_t L;
  int ng;//0/1/2/3: gamma/neutron/cparticle/nohit
  Double_t TOF, nTOF;
  Double_t tu, td;
  Double_t Qu, Qd;
  //Int_t itu, itd;
  //Int_t iQu, iQd;
    
  Double_t tu_off=5.5;//time offset
  Double_t td_off=20.4;//time offset
 
  // 将变量分支添加到tree结构中,第一个参数为变量名称，第二个为上面定义的变量地址，第三个为变量的类型说明，D表示Double_t。
  opt->Branch("x", &x, "x/D");//position of neutron
  opt->Branch("E", &E, "E/D");//energy of gamma or neutron
  opt->Branch("TOF", &TOF, "TOF/D");//time of flight
  opt->Branch("L",&L,"L/D");
  opt->Branch("nTOF",&nTOF,"nTOF/D");//TOF from exp. data
  opt->Branch("ng", &ng, "ng/I");//1/0 : neutron/gamma  
    
  opt->Branch("tu", &tu, "tu/D");//time of upper side
  opt->Branch("td", &td, "td/D");//time of bottom side
  opt->Branch("Qu", &Qu, "Qu/D");//Q 
  opt->Branch("Qd", &Qd, "Qd/D");//Q  
  //opt->Branch("tu", &itu, "tu/D");//time of upper side
  //opt->Branch("td", &itd, "td/D");//time of bottom side
  //opt->Branch("Qu", &iQu, "Qu/D");//Q 
  //opt->Branch("Qd", &iQd, "Qd/D");//Q  

  // 将变量分支添加到tree结构中,第一个参数为变量名称，第二个为上面定义的变量地址，第三个为变量的类型说明，D表示Double_t。
  opt->Branch("x", &x, "x/D");//position of neutron
  opt->Branch("E", &E, "E/D");//energy of gamma or neutron
  opt->Branch("TOF", &TOF, "TOF/D");//time of flight
  opt->Branch("L",&L,"L/D");
  opt->Branch("nTOF",&nTOF,"nTOF/D");//TOF from exp. data
  opt->Branch("ng", &ng, "ng/I");//1/0 : neutron/gamma  
  opt->Branch("tu", &tu, "tu/D");//time of upper side
  opt->Branch("td", &td, "td/D");//time of bottom side
  opt->Branch("Qu", &Qu, "Qu/D");//Q 
  opt->Branch("Qd", &Qd, "Qd/D");//Q  
  
  
  TRandom3 *gr=new TRandom3(0);
  // 循环，逐事件往tree结构里添加对应分支信息。
  for(int i=0;i<1000000;i++){
    x=gr->Uniform(-nL, nL);
    Double_t D=nD+gr->Uniform(-0.5,0.5)*nTh;
    L=TMath::Sqrt(D*D+x*x)*0.01;//m, flight path
    
    if(gr->Uniform()<ngeff) {
      if(gr->Uniform() < rgamma) { //gamma
	ng=0;
	E=Eg0;
	TOF=3*L;
      }
      else {  //neutron
        ng=1;
        E=gr->Gaus(En0, Ens); // neutron
        TOF=72./TMath::Sqrt(E)*L;
      }
    }
    else if(gr->Uniform()<cpeff) {//charged particle
      ng=2;
      E=gr->Gaus(Ec0, Ecs); // neutron
      TOF=72./TMath::Sqrt(E)*L;
    }
    else {//no particle hit neutron detector
      ng=3;
    }
  
    if(ng==3) {//
      tu=TDCoverflow;
      td=TDCoverflow;
      Qu=ADCuPed+gr->Gaus(0,ADCnoise);
      Qd=ADCdPed+gr->Gaus(0,ADCnoise);
    }
    else {
      tu=TOF+(nL-x)/vsc+gr->Gaus(0,ts)+tu_off;
      td=TOF+(nL+x)/vsc+gr->Gaus(0,ts)+td_off;
      tu *=TDCgain;
      td *=TDCgain;
      Double_t Q=E*ADCgain*gr->Uniform();//neutron &gamma
      if(ng==2) Q=E*ADCgain;//charged particle
      Qu=Q*TMath::Exp(-(nL-x)/lambda);
      Qu=gr->Gaus(Qu,Qs*Qu)+ADCuPed+gr->Gaus(0,ADCnoise);
      Qd=Q*TMath::Exp(-(nL+x)/lambda);
      Qd=gr->Gaus(Qd,Qs*Qd)+ADCdPed+gr->Gaus(0,ADCnoise);
       if(gr->Uniform()<0.1) {
           tu=TDCoverflow;
           Qu=ADCuPed+gr->Gaus(0,ADCnoise);
       }
       if(gr->Uniform()>0.9) {
           td=TDCoverflow;
           Qd=ADCdPed+gr->Gaus(0,ADCnoise);
    }
    if(tu>TDCoverflow) tu=TDCoverflow;
    if(tu>TDCoverflow) tu=TDCoverflow;
    if(Qu>ADCoverflow) Qu=ADCoverflow;
    if(Qd>ADCoverflow) Qd=ADCoverflow;
    //itu=tu;
     //itd=td;
     //iQu=Qu;
     //iQd=Qd;
    opt->Fill();
  }
  // 将数据写入root文件中
  opt->Write();
  opf->Close();
}

```

In [1]:
%jsroot on

In [2]:
TFile *ipf=new TFile("treeover.root");//root -l tree.root


In [3]:
TCanvas *c1=new TCanvas();//* 在ROOT环境下可省略
c1->Clear();//* 在ROOT环境下可省略


In [4]:
tree->Draw("tu>>htu(500,0,5500)");
TH1D *htu=(TH1D*)gROOT->FindObject("htu");
tree->Draw("td>>htd(500,0,5500)");
TH1D *htd=(TH1D*)gROOT->FindObject("htd");
htd->SetLineColor(kGreen);

htu->Draw();
htd->Draw("same");
c1->SetLogy();
c1->Draw();

In [5]:
tree->Draw("tu:td>>(500,0,5500,500,0,5500)","","colz");
gPad->SetLogz();
gPad->SetLogy(0);
c1->Draw();//* 在ROOT环境下可省略

In [6]:
tree->Draw("tu-td");
gPad->SetLogy();
c1->Draw();

In [7]:
tree->Draw("tu-td","tu<5000 && td<5000");
gPad->SetLogy();
c1->Draw();

In [8]:
tree->Draw("Qu>>hng(5500,0,5500)");
TH1D *hng=(TH1D*)gROOT->FindObject("hng");
hng->SetLineColor(kBlack);

tree->Draw("Qu>>hng1(5500,0,5500)","ng==0 || ng==1");
TH1D *hng1=(TH1D*)gROOT->FindObject("hng1");
hng1->SetLineColor(kRed);

tree->Draw("Qu>>hng2(5500,0,5500)","ng==2");
TH1D *hng2=(TH1D*)gROOT->FindObject("hng2");
hng2->SetLineColor(kGreen);


tree->Draw("Qu>>hng3(5500,0,5500)","ng==3");
TH1D *hng3=(TH1D*)gROOT->FindObject("hng3");
hng3->SetLineColor(kBlue);
gPad->SetLogy();
hng->Draw();
hng1->Draw("same");
hng2->Draw("same");
hng3->Draw("same");
c1->Draw();

In [9]:
hng->Fit("gaus","",0,150);

 FCN=618602 FROM MIGRAD    STATUS=CONVERGED     292 CALLS         293 TOTAL
                     EDM=0.0166021    STRATEGY= 1  ERROR MATRIX UNCERTAINTY   6.8 per cent
  EXT PARAMETER                                   STEP         FIRST   
  NO.   NAME      VALUE            ERROR          SIZE      DERIVATIVE 
   1  Constant     1.87634e+02   4.08838e-01   0.00000e+00  -1.87071e-02
   2  Mean         1.14712e+03   2.02315e+00   0.00000e+00  -1.28422e-03
   3  Sigma        9.18945e+02   2.01593e+00  -0.00000e+00   5.34785e+02


In [10]:
tree->Draw("Qu:Qd>>(500,0,5500,500,0,5500)","","colz");
c1->SetLogz();
c1->SetLogy(0);
c1->Draw();//* 在ROOT环境下可省略

In [11]:
tree->Draw("sqrt(Qu*Qd)");
gPad->SetLogy();
c1->Draw();

In [12]:
tree->SetAlias("qu","Qu-100");
tree->SetAlias("qd","Qd-120");
TCut cq= "Qu>150 && Qu<4900 && Qd>150&&Qd<4900";//>ped+3*sigma
tree->Draw("sqrt(qu*qd)",cq);
gPad->SetLogy();
c1->Draw();

In [16]:
tree->Draw("tu-td:log(qu/qd)",cq&&"tu<5000 && td<5000","colz");
c1->SetLogz();
c1->SetLogy(0);
c1->Draw();